In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
df = pd.read_csv("./labeled_data.csv")
df.head()

,Label,0,1,2,3,4,5,6,7,8,...,191834,191835,191836,191837,191838,191839,191840,191841,191842,191843
0,0,149.714737,149.718140,149.720032,149.715073,149.709106,149.704453,149.699890,149.699188,149.697052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,149.700302,149.691193,149.687744,149.688904,149.692169,149.694214,149.694397,149.697479,149.696991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,149.646027,149.633041,149.626465,149.626328,149.632492,149.642410,149.658417,149.669891,149.681183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,149.640610,149.646973,149.655563,149.663971,149.670441,149.674347,149.672531,149.667542,149.661957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,149.682312,149.681534,149.680725,149.674713,149.664841,149.656952,149.649673,149.642334,149.634903,...,149.679962,149.679382,149.677216,149.674072,149.668625,149.668945,149.67038,149.673447,149.674194,149.676224


In [38]:
df = df.fillna(0)
df.head()

,Label,0,1,2,3,4,5,6,7,8,...,191834,191835,191836,191837,191838,191839,191840,191841,191842,191843
0,0,149.714737,149.718140,149.720032,149.715073,149.709106,149.704453,149.699890,149.699188,149.697052,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,0,149.700302,149.691193,149.687744,149.688904,149.692169,149.694214,149.694397,149.697479,149.696991,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0,149.646027,149.633041,149.626465,149.626328,149.632492,149.642410,149.658417,149.669891,149.681183,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,0,149.640610,149.646973,149.655563,149.663971,149.670441,149.674347,149.672531,149.667542,149.661957,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,0,149.682312,149.681534,149.680725,149.674713,149.664841,149.656952,149.649673,149.642334,149.634903,...,149.679962,149.679382,149.677216,149.674072,149.668625,149.668945,149.67038,149.673447,149.674194,149.676224


In [44]:
X = np.array(df.drop('Label', axis=1))
Y = np.array(df['Label'])

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [45]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [48]:
X_train_scaled = X_train_scaled.reshape(-1, 438, 438, 1)
X_test_scaled = X_test_scaled.reshape(-1, 438, 438, 1)

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [54]:
input_shape = (438, 438, 1)

model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Flatten and Dense Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [55]:
history = model.fit(X_train_scaled, Y_train,
          batch_size=16,
          epochs=20,
          verbose=1)

ResourceExhaustedError: OOM when allocating tensor with shape[54194048,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node dense_2/kernel/Initializer/random_uniform/RandomUniform (defined at \AppData\Local\Temp\ipykernel_6596\3879469996.py:33) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'dense_2/kernel/Initializer/random_uniform/RandomUniform':
  File "\miniconda3\envs\mlcricket\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "\miniconda3\envs\mlcricket\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "\miniconda3\envs\mlcricket\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "\miniconda3\envs\mlcricket\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  F